In [4]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 527.5 kB/s eta 0:00:0000:0100:01


In [5]:
import duckdb

In [7]:
# Create an in-memory database connection
conn = duckdb.connect(database=':memory:')

# Create a table
conn.execute("""
CREATE TABLE users (
    id INTEGER,
    name STRING,
    age INTEGER
)
""")

# Insert some data
conn.execute("""
INSERT INTO users VALUES 
(1, 'Alice', 30),
(2, 'Bob', 25),
(3, 'Charlie', 35)
""")

# Query the table
result = conn.execute("SELECT * FROM users").fetchall()

# Print the result
print(result)

[(1, 'Alice', 30), (2, 'Bob', 25), (3, 'Charlie', 35)]


In [20]:
import duckdb
conn = duckdb.connect(database=':memory:')
conn.execute("""
INSTALL fts;
LOAD fts;
CREATE TABLE documents (
    document_identifier VARCHAR,
    text_content VARCHAR,
    author VARCHAR,
    doc_version INTEGER
);
""")

In [21]:
conn.execute("""
INSERT INTO documents
    VALUES ('doc1',
            'The mallard is a dabbling duck that breeds throughout the temperate.',
            'Hannes Mühleisen',
            3),
           ('doc2',
            'The cat is a domestic species of small carnivorous mammal.',
            'Laurens Kuiper',
            2
           );
""")

In [22]:
conn.execute("""PRAGMA create_fts_index(
    'documents', 'document_identifier', 'text_content', 'author'
);""")

In [30]:
conn.execute("""
SELECT document_identifier, text_content, score
FROM (
    SELECT *, fts_main_documents.match_bm25(
        document_identifier,
        'Muhleisen',
        fields := 'author'
    ) AS score
    FROM documents
) sq
WHERE score IS NOT NULL
  AND doc_version > 2
ORDER BY score DESC;
""").fetchall()

[('doc1',
  'The mallard is a dabbling duck that breeds throughout the temperate.',
  0.3094700890003546)]

In [31]:
conn.execute("""
SELECT document_identifier, text_content, score
FROM (
    SELECT *, fts_main_documents.match_bm25(
        document_identifier,
        'small cats'
    ) AS score
    FROM documents
) sq
WHERE score IS NOT NULL
ORDER BY score DESC;
""").fetchall()

[('doc2',
  'The cat is a domestic species of small carnivorous mammal.',
  0.5860760977528838)]